In [2]:
from scipy.sparse import csr_matrix, lil_matrix, load_npz, save_npz, hstack
import numpy as np 
import pandas as pd 
from tqdm import tqdm 

In [5]:
path = '.../y2_ext_noaux/matrices/cls/cls_T10_y.npz'
y = load_npz(path)

In [ ]:
y

In [ ]:
path = '../data/baseline_plus_aux_data/matrices/cls/cls_T10_y.npz'
y_aux = load_npz(path)

In [ ]:
y_aux

In [ ]:
# the aux tasks have the highest index numbers 
y_out = y_aux[:,:y.shape[1]]

In [ ]:
y_out

In [ ]:
path = '../data/baseline_plus_aux_data/matrices/cls/cls_T10_y_noaux.npz'
save_npz(path, y_out)

In [ ]:
## Redo here for the model training

In [ ]:
y_aux_empty = csr_matrix((y_aux.shape[0],y_aux.shape[1] - y_out.shape[1]))

In [ ]:
hstack((y_out,y_aux_empty))

In [ ]:
path = '../data/baseline_plus_aux_data/matrices/cls/cls_T10_y_noaux.npz'
save_npz(path,hstack((y_out,y_aux_empty)))

In [ ]:
# Generate the y-sparse matrix for inference 
# Main tasks + fold 0 to limit the amount of predictions

In [ ]:
path = '../data/baseline_plus_aux_data/matrices/cls/cls_T11_fold_vector.npy'
folds = np.load(path)

In [ ]:
len(folds)

In [ ]:
y_aux

In [ ]:
path = '../analysis/cp/summary_eps_0.05.csv'
cp = pd.read_csv(path)

In [ ]:
col_idxs = list(cp.query('validity_0>0').query('validity_1>0')['index'])

In [ ]:
y_sparse_main_fold0 = lil_matrix(y_aux.shape)

In [ ]:
y_sparse_main_fold0

In [ ]:
folds == 0

In [ ]:
for col in tqdm(col_idxs) :
    y_sparse_main_fold0[folds == 0,col] = 1

In [ ]:
y_sparse_main_fold0

In [ ]:
path = '../data/baseline_plus_aux_data/matrices/cls/y_sparse_main_fold0.npy'
save_npz(path,csr_matrix(y_sparse_main_fold0))

In [ ]:
pwd